In [5]:
import sys
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")

from MARScore.utils import * 
from sklearn.model_selection import RandomizedSearchCV
import hdbscan
from sklearn.metrics import make_scorer
from random import seed
from datasets_loaders.loaders import load_billsum

In [9]:
billsum = load_billsum()
subset = billsum.iloc[:300, :]

In [7]:
def HDBScanFinetune(corpus, 
                    min_samples=[10,30,50,60,100], 
                    min_cluster_size=[100,200,300,400,500,600],
                    cluster_selection_method=['eom','leaf'],
                    metric=['euclidean','manhattan'],
                    seed=seed(0), 
                    verbose=True):
    #creation of embeddings
    for indiv in corpus:
        o, l = tokenizeCorpus(indiv)
        v = vectorizeCorpus(o)
        v, l = cleanAll(v, l)
    
    #finetuning
    hdb = hdbscan.HDBSCAN(gen_min_span_tree=True).fit(v)

    # specify parameters and distributions to sample from
    param_dist = {'min_samples': min_samples,
                'min_cluster_size':min_cluster_size,  
                'cluster_selection_method':cluster_selection_method,
                'metric':metric  
                }

    #validity_scroer = "hdbscan__hdbscan___HDBSCAN__validity_index"
    validity_scorer = make_scorer(hdbscan.validity.validity_index,greater_is_better=True)


    n_iter_search = 20
    random_search = RandomizedSearchCV(hdb
                                    ,param_distributions=param_dist
                                    ,n_iter=n_iter_search
                                    ,scoring=validity_scorer
                                    ,random_state=seed)
    random_search.fit(v)

    if verbose:
        print(f"Best Parameters {random_search.best_params_}")
        print(f"DBCV score :{random_search.best_estimator_.relative_validity_}")
    return {"best_params": random_search.best_params_, "dbcv_score":random_search.best_estimator_.relative_validity_}

In [ ]:
HDBScanFinetune(billsum["text"].to_list())